In [2]:
%pip install opentrons

  Using cached opentrons-8.5.0-py2.py3-none-any.whl.metadata (5.1 kB)
  Using cached opentrons_shared_data-8.5.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached aionotify-0.3.1-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/959.7 kB ? eta -:--:--
   ---------------------------------------- 959.7/959.7 kB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------- -- 1.8/2.0 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install prefect

  Using cached prefect-3.4.8-py3-none-any.whl.metadata (13 kB)
  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached alembic-1.16.4-py3-none-any.whl.metadata (7.3 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached apprise-1.9.3-py3-none-any.whl.metadata (53 kB)
  Using cached asgi_lifespan-2.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached asyncpg-0.30.0-cp311-cp311-win_amd64.whl.metadata (5.2 kB)
  Using cached cachetools-6.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached coolname-2.2.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached cryptography-45.0.5-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached dateparser-1.2.2-py3-none-any.whl.metadata (29 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached exceptiongroup-1.3.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadat

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentrons 8.5.0 requires anyio<4.0.0,>=3.6.1, but you have anyio 4.9.0 which is incompatible.
opentrons 8.5.0 requires jsonschema<4.18.0,>=3.0.1, but you have jsonschema 4.24.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    os.system("wget -q https://raw.githubusercontent.com/AccelerationConsortium/ac-training-lab/main/src/ac_training_lab/ot-2/_scripts/ac_6_tuberack_15000ul.json")
    os.system("wget -q https://raw.githubusercontent.com/AccelerationConsortium/ac-training-lab/main/src/ac_training_lab/ot-2/_scripts/ac_color_sensor_charging_port.json")
    print("JSON labware files downloaded for Colab")
else:
    print("Not running in Colab, skipping download")


Not running in Colab, skipping download


In [5]:
# source: https://support.opentrons.com/s/article/Running-the-robot-using-Jupyter-Notebook

# import opentrons.execute
import opentrons.simulate  # if you want to simulate the protocol first

# from opentrons.config import advanced_settings

# print(
#     "useProtocolEngine =",
#     advanced_settings.get_adv_setting("useProtocolEngine", "OT-2"),
# )


# This is where you establish the API version for executing a protocol
# protocol = opentrons.execute.get_protocol_api('2.11')
# to simulate the protocol, replace opentrons.execute with opentrons.simulate
# like so
protocol = opentrons.simulate.get_protocol_api("2.16")

# Must home before running
protocol.home()

# Labware
plate = protocol.load_labware("nest_96_wellplate_200ul_flat", "2")

tiprack_1 = protocol.load_labware("opentrons_96_tiprack_1000ul", location="1")

# Pipette
p1000 = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tiprack_1])

# Protocol steps
p1000.transfer(100, plate["A1"], plate["A2"])

# Output steps in protocol
for line in protocol.commands():
    print(line)

C:\Users\86173\.opentrons\robot_settings.json not found. Loading defaults
Deck calibration not found.
C:\Users\86173\.opentrons\deck_calibration.json not found. Loading defaults


Transferring 100.0 from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 to A2 of NEST 96 Well Plate 200 µL Flat on slot 2
Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1
Aspirating 100.0 uL from A1 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dispensing 100.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on slot 2 at 274.7 uL/sec
Dropping tip into Trash Bin on slot 12


In [15]:
import json

import opentrons.simulate
from prefect import flow, task

protocol = plate = reservoir = tiprack_1 = tiprack_2 = p300 = None


protocol = opentrons.simulate.get_protocol_api("2.12")

protocol.home()  # home the robot

# Define labware and pipettes
# Load wireless charging port
with open(
    # "/var/lib/jupyter/notebooks/ac_color_sensor_charging_port.json"
    "ac_color_sensor_charging_port.json",
    encoding="utf-8" 
) as labware_file1:
    labware_def1 = json.load(labware_file1)
    tiprack_2 = protocol.load_labware_from_definition(labware_def1, 10)

# Load 3x2 vials rack
with open(
    # "/var/lib/jupyter/notebooks/ac_6_tuberack_15000ul.json"
    "ac_6_tuberack_15000ul.json",
    encoding="utf-8" 
) as labware_file2:
    labware_def2 = json.load(labware_file2)
    reservoir = protocol.load_labware_from_definition(labware_def2, 3)

# Load other labwares from Opentrons's labware library
plate = protocol.load_labware(load_name="corning_96_wellplate_360ul_flat", location=1)
tiprack_1 = protocol.load_labware(load_name="opentrons_96_tiprack_300ul", location=9)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2", mount="right", tip_racks=[tiprack_1]
)

p300.well_bottom_clearance.dispense = 8

print("Labwares loaded")


@task
def mix_color(R, Y, B, mix_well):
    """Mix colors with specified RGB values into a well"""
    total = R + Y + B
    if total > 300:
        raise ValueError("The sum of the proportions must be not greater than 300")

    position = [
        "B1",
        "B2",
        "B3",
    ]  # R,Y,B paint vial on the position of the 3x2 vials rack
    portion = {"B1": R, "B2": Y, "B3": B}

    red_volume = R
    yellow_volume = Y
    blue_volume = B
    color_volume = {"B1": red_volume, "B2": yellow_volume, "B3": blue_volume}

    # assert all(i is not None for i in [p300, tiprack_1, reservoir, plate])
    # doing this for pylance
    assert p300 is not None
    assert tiprack_1 is not None
    assert reservoir is not None
    assert plate is not None

    for pos in position:
        if float(portion[pos]) != 0.0:  # if zero, skip this color aspiration process
            p300.pick_up_tip(tiprack_1[pos])
            p300.aspirate(color_volume[pos], reservoir[pos])
            p300.dispense(color_volume[pos], plate[mix_well])
            p300.default_speed = 100
            # reduce pipette speed to prevent droplets falling, speed unit: mm/sec
            p300.blow_out(reservoir["A1"].top(z=-5))
            p300.default_speed = (
                400  # reset pipette speed, speed unit: mm/sec, 400 is default value
            )
            p300.drop_tip(tiprack_1[pos])

    print(f"Mixed R:{R}, Y:{Y}, B:{B} in well {mix_well}")


@task
def move_sensor_to_measurement_position(mix_well):
    """Move sensor to measurement position"""
    # Move sensor to measurement position
    assert p300 is not None
    assert tiprack_2 is not None
    assert plate is not None
    p300.pick_up_tip(tiprack_2["A2"])
    # p300.move_to(tiprack_2["A2"].top())
    p300.move_to(plate[mix_well].top(z=-1.3))  # z=-1.3 to get sensor closer to the well
    print("Sensor is now in position for measurement")


@task
def move_sensor_back():
    """Move sensor back to charging position"""
    assert p300 is not None
    assert tiprack_2 is not None
    p300.drop_tip(tiprack_2["A2"].top(z=-80))
    # p300.move_to(tiprack_2["A2"].top(z=-80))
    print("Sensor moved back to charging position")


# `SequentialTaskRunner` is used to run tasks sequentially (sync).
@flow(name="main_flow")
def main_flow(R: int = 100, Y: int = 100, B: int = 100, mix_well: str = "A1"):
    """
    Main flow is basically a wrapper for `mix_color` task.
    This is because `mix_color` is now a `task` and can be used in a `flow`.
    """
    # init()
    mix_color(R, Y, B, mix_well)
    move_sensor_to_measurement_position(mix_well)
    move_sensor_back()


if __name__ == "__main__":
    ## Below cannot happen in Jupyter Notebook, but can be used in Python script.
    # main_flow.serve(
    #     name="my-first-deployment",
    #     tags=["onboarding"],
    #     parameters={"goodbye": True},
    #     interval=60,
    # )

    ## That's why we use `main_flow()` directly here.
    main_flow()

09:38:03.070 | WARNING | opentrons.config.robot_configs - C:\Users\86173\.opentrons\robot_settings.json not found. Loading defaults

09:38:03.073 | WARNING | opentrons.calibration_storage.ot2.deck_attitude - Deck calibration not found.

09:38:03.075 | WARNING | opentrons.config.robot_configs - C:\Users\86173\.opentrons\deck_calibration.json not found. Loading defaults

Labwares loaded


09:38:03.417 | INFO    | Flow run 'woodoo-giraffe' - Beginning flow run 'woodoo-giraffe' for flow 'main_flow'

...

Sensor is now in position for measurement


09:38:03.975 | INFO    | Task run 'move_sensor_to_measurement_position-7c7' - Finished in state Completed()

Sensor moved back to charging position


09:38:04.248 | INFO    | Task run 'move_sensor_back-99c' - Finished in state Completed()

09:38:04.273 | INFO    | Flow run 'woodoo-giraffe' - Finished in state Completed()

## Code Backup

In [25]:
# # from prefect.runner.storage import GitRepository, GitCredentials

# from prefect import serve

# from demo import demo_flow
# from final_state_from_task import running_flow

# from ot2mqtt import mix_color, move_sensor_to_measurement_position, move_sensor_back

# serve(
#     mix_color.to_deployment("mix-color"),  # generates OpenAPI schema
#     move_sensor_to_measurement_position.to_deployment(
#         "move-sensor-to-measurement-position"
#     ),
#     move_sensor_back.to_deployment("move-sensor-back"),
# )

In [26]:
# test= running_flow.from_source(
#         source=GitRepository(
#             url="https://github.com/masonmenges/mm2-sanbox.git"
#             ),
#         entrypoint="flows/final_state_from_task.py:runni
# )

https://docs.prefect.io/v3/deploy/deployment-versioning#git-hub-actions